In [1]:
# import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install python-dotenv --upgrade
# !{sys.executable} -m pip install pandas --upgrade
# !{sys.executable} -m pip install gql[requests]

## Imports

In [2]:
# For caching
from functools import cache, lru_cache

# GQL clinet libraries
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# Date calculations
from datetime import datetime, timedelta
import time

# For dataframe
import pandas as pd

# For plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Display numbers in human read format
from numerize import numerize

# For regular expressions
import re

## Constants

In [3]:
# The graph.com endpoint for ENS
GRAPHQL_ENDPOINT = 'https://api.morpho.org/graphql'

# How many days back
DAYS = 90

## Initialize GraphQL client

In [4]:
# Create a transport object
_transport = RequestsHTTPTransport(url=GRAPHQL_ENDPOINT, verify=True, retries=3)
# Create a GraphQL client
client = Client(transport=_transport, fetch_schema_from_transport=True)

## Query: Morpho Chains

In [5]:
@cache
def query_morpho_chains() -> str:
    query = gql('''
        query Chain {
          chains {
            id
          }
        }    
    ''')
    return client.execute(query)

## Morpho chains as a dictionary

In [6]:
data = query_morpho_chains()
chains = [x['id'] for x in data['chains']]
chains

[1, 8453, 57073, 137, 130, 10]

## Query: Morpho Blues TVL

In [7]:
@lru_cache
def query_morpho_blue_tvl(chain_id:int, start_time:int) -> list:
    query = gql('''
        query MorphoBlues(
          $first: Int
          $options: TimeseriesOptions
          $where: MorphoBlueFilters
        ) {
          morphoBlues(first: $first, where: $where) {
            items {
              chain {
                network
              }
              historicalState {
                tvlUsd(options: $options) {
                  x
                  y
                }
              }
            }
          }
        }
    ''')
    # Execute the query with variables
    params = {
      'options': {
        'interval': 'DAY',
        'startTimestamp': start_time
      },
      'where': {
        'chainId_in': chain_id
      },
    }
    json_data = client.execute(query, variable_values=params)

    # Short cut to items
    items = json_data['morphoBlues']['items']

    # Check for empty result
    if len(items) == 0:
        return []
    tvl_usd = items[0]['historicalState']['tvlUsd']
    network = items[0]['chain']['network']
    # Enrich the response by adding network to the response
    response = [{'timestamp': x['x'], 'tvl':x['y'], 'chain': network} for x in tvl_usd]
    return response

## Morpho Blue TVL as a DF

In [8]:
def df_morpho_blue_tvl(chains:[int]) -> pd.DataFrame:
    # This is the startig date (DAYS befoe the current date)
    d = datetime.today() - timedelta(DAYS)
    # Reset time components - this will allow us to cache the result for a day
    dt = d.replace(hour=0, minute=0, second=0, microsecond=0)
    start_time = int(dt.timestamp())
    # DF to hold tvl values for each chain
    df = pd.DataFrame()
    for chain_id in chains:
        # List of TVL records
        tvl = query_morpho_blue_tvl(chain_id=chain_id, start_time=start_time)

        # TVL can be empty
        if len(tvl) == 0:
            continue

        # For the first chain, DF is empty. Create one using tvl
        if df.empty:
            # Create a new one
            df = pd.DataFrame(tvl)
        else:
            # Append to the existing DF
            df = pd.concat([df, pd.DataFrame(tvl)], ignore_index=True)
    # Convert the timestamp to date format
    df['date'] = pd.to_datetime(df['timestamp'], unit='s')
    # Drop the timestamp column.
    df = df.drop('timestamp', axis=1)
    return df

## Plot Morpho Blue TVL

In [9]:
def plot_morpho_blue_tvl(chains:[int]) -> None:
    df = df_morpho_blue_tvl(chains)
    fig = px.area(df, x='date', y='tvl', color='chain')
    fig.update_layout(
        title=dict(text=f'Morpho Blue TVL from {df['date'].iloc[-1].strftime('%Y-%m-%d')} \
    to {df['date'][0].strftime('%Y-%m-%d')}'),
        xaxis=dict(title=dict(text='Date')),
        yaxis=dict(title=dict(text='TVL - USD')),
        template='plotly_dark')
    fig.show()

In [10]:
# plot_morpho_blue_tvl(chains=chains)

## Query: Morpho Blues State

In [11]:
def query_morpho_blue_state() -> str:
    query = gql('''
        query Chain {
          morphoBlues {
            items {
              state {
                timestamp
                tvlUsd
                totalBorrowUsd
                totalSupplyUsd
                totalDepositUsd
                totalCollateralUsd
              }
              chain {
                network
              }
            }
          }
        }
    ''')
    return client.execute(query)

## Blue State as DF

In [12]:
def df_morpho_blue_state() -> pd.DataFrame:
    # Query state data
    state_data = query_morpho_blue_state()

    # Store chain states
    chain_states = []
    for item in state_data['morphoBlues']['items']:
        row = item['state']
        row['chain'] = item['chain']['network']
        chain_states.append(row)
    return pd.DataFrame(data=chain_states)

## Plot Morpho Blue State

In [13]:
def plot_morpho_blue_state() -> None:
    # Query state data
    df = df_morpho_blue_state()
    assert not df.empty, "Morpho Blue state can't be empty"
    
    # Start date for the chart title
    state_date = pd.to_datetime(df['timestamp'].iloc[0], unit='s')
    
    fig = make_subplots(rows=1, cols=5, specs=[[{'type':'domain'}, {'type':'domain'},
                                                {'type':'domain'}, {'type':'domain'}, {'type':'domain'}]])
    
    fig.add_trace(go.Pie(labels=df['chain'], values=df['tvlUsd'], name='TVL'), 1,1)
    fig.add_trace(go.Pie(labels=df['chain'], values=df['totalBorrowUsd'], name='Total Borrow'), 1,2)
    fig.add_trace(go.Pie(labels=df['chain'], values=df['totalSupplyUsd'], name='Total Supply'), 1,3)
    fig.add_trace(go.Pie(labels=df['chain'], values=df['totalDepositUsd'], name='Total Deposit'), 1,4)
    fig.add_trace(go.Pie(labels=df['chain'], values=df['totalCollateralUsd'], name='Total Collateral'), 1,5)
    
    # Use `hole` to create a donut-like pie chart
    fig.update_traces(hoverinfo='label+percent+name', textfont_size=14,
                      marker=dict(line=dict(color='#000000', width=1)))
    fig.update_layout(
        title=dict(text=f'<b>Morpho Blues State as at {state_date.strftime('%Y-%m-%d %H:%M:%S')}</b>',
                   font=dict(size=20, color='cornflowerblue'), x=0.5, y=0.9, xanchor='center'),
        # Add annotations in the center of the donut pies.
        annotations=[dict(text='<b>TVL</b>', x=sum(fig.get_subplot(1, 1).x)/2, y=1, 
                          font=dict(size=16, color='mediumturquoise'), showarrow=False, xanchor='center'),
                     dict(text='<b>Total Borrow<b>', x=sum(fig.get_subplot(1, 2).x)/2, y=1,
                          font=dict(size=16, color='mediumturquoise'), showarrow=False, xanchor='center'),
                    dict(text='<b>Total Supply<b>', x=sum(fig.get_subplot(1, 3).x)/2, y=1,
                          font=dict(size=16, color='mediumturquoise'), showarrow=False, xanchor='center'),
                    dict(text='<b>Total Deposit<b>', x=sum(fig.get_subplot(1, 4).x)/2, y=1,
                          font=dict(size=16, color='mediumturquoise'), showarrow=False, xanchor='center'),
                    dict(text='<b>Total Collateral<b>', x=sum(fig.get_subplot(1, 5).x)/2, y=1,
                          font=dict(size=16, color='mediumturquoise'), showarrow=False, xanchor='center')],
        template='plotly_dark')
    fig.show()

In [14]:
# plot_morpho_blue_state()

## Query: Morpho Markets

In [15]:
def query_morpho_markets() -> str:
    query = gql('''
        query MorphoBlue(
          $orderBy: MarketOrderBy
          $orderDirection: OrderDirection
          $first: Int
          $where: MarketFilters
        ) {
          markets(
            orderBy: $orderBy
            orderDirection: $orderDirection
            first: $first
            where: $where
          ) {
            items {
              loanAsset {
                chain {
                  network
                }
                symbol
              }
              collateralAsset {
                chain {
                  network
                }
                symbol
              }
              state {
                timestamp
                supplyAssetsUsd
                borrowAssetsUsd
                collateralAssetsUsd
                totalLiquidityUsd
                liquidityAssetsUsd
                utilization
                borrowApy
                supplyApy
              }
              lltv
            }
          }
        }
    ''')
    # Execute the query with variables
    params = {
      "orderBy": "SupplyAssetsUsd",
      "orderDirection": "Desc",
      "first": 10,
      "where": {
        "credoraRiskScore_gte": 0,
        "lltv_gte": 100000000000000000
      }
    }    
    # print('in TVL query')
    json_data = client.execute(query, variable_values=params)
    return json_data

## Morpho Markets as a DF

In [16]:
def df_morpho_markets() -> pd.DataFrame:
    markets_data = query_morpho_markets()
    # List to collect markets data
    markets_list = []
    for item in markets_data['markets']['items']:
        collateral_chain = item['collateralAsset']['chain']['network']
        loan_chain = item['loanAsset']['chain']['network']
        assert collateral_chain == loan_chain, "Collateral/Loan chains don't match"

        collateral_asset = item['collateralAsset']['symbol']
        loan_asset = item['loanAsset']['symbol']
        lltv = int(item['lltv'])/pow(10,16)
        
        row = item['state']
        row['market'] = f'{collateral_asset}/{loan_asset} ({lltv}%)'
        row['chain'] = collateral_chain
        markets_list.append(row)

    df = pd.DataFrame(data=markets_list)
    # Arrange the columns; move markets and chain to the front
    cols = list(df.columns)
    df = df.loc[:, cols[-2:] + cols[0:-2]]
    # Make numbers human readable
    for name in list(df.columns)[3:8]:
        df[name] = df[name].apply(lambda x: float(x)/1000000)
    # Round up the APYs
    for name in list(df.columns)[-2:]:
        df[name] = df[name].apply(lambda x: round(x,4))
    df['utilization'] = df['utilization'].apply(lambda x: round(float(x),4)*100)
    return df

In [17]:
def make_pretty(styler):
    # Column formatting
    styler.format(
        {'Supply Assets (mm)':'${:.2f}','Borrow Assets (mm)':'${:.2f}',
         'Collateral Assets (mm)':'${:.2f}','Total Liquidity (mm)':'${:.2f}',
         'Liquidity Assets (mm)':'${:.2f}','Utilization':'{:.2f}%',
         'Borrow APY': '{:.2f}%', 'Supply APY': '{:.2f}%'
        }
    )
    # Set the bar visualization
    styler.bar(subset=['Utilization'], color='green', height=60)

    # Set background gradients
    styler.background_gradient(subset=['Liquidity Assets (mm)'], cmap='Greens')
    
    # Grid
    styler.set_properties(**{'border': '0.1px solid black'})

     # Left text alignment for some columns
    styler.set_properties(subset=['Market', 'Chain'], **{'text-align': 'left'})

    styler.set_table_styles(
        [{'selector': 'th.col_heading', 'props': 'text-align: center'},
         {'selector': 'caption', 'props': [('text-align', 'center'),
                                           ('font-size', '14pt'), ('font-weight', 'bold')]}])
    styler.hide(axis='index')
    return styler

## Display Morpho Markets

In [18]:
def display_morpho_markets(df:pd.DataFrame) -> None:
    # Format the timestamp from the first item from the timestamp
    ts = time.strftime('%Y-%m-%d %H:%M:%S %Z', time.gmtime(df['timestamp'][0].item()))
    # Will drop the timstamp column as we already have the ts from the first item
    df = df.drop('timestamp', axis=1)
    new_cols = ['Market', 'Chain', 'Supply Assets (mm)', 'Borrow Assets (mm)', 'Collateral Assets (mm)',
                'Total Liquidity (mm)', 'Liquidity Assets (mm)', 'Utilization', 'Borrow APY', 'Supply APY']
    df = df.set_axis(new_cols, axis=1)
    # Add table caption and styles to DF
    return df.style.pipe(make_pretty).set_caption(f'Morpho Markets as at {ts}')

In [19]:
df = df_morpho_markets()
display_morpho_markets(df=df)

Market,Chain,Supply Assets (mm),Borrow Assets (mm),Collateral Assets (mm),Total Liquidity (mm),Liquidity Assets (mm),Utilization,Borrow APY,Supply APY
cbBTC/USDC (86.0%),base,$430.79,$387.78,$811.65,$43.31,$43.00,90.02%,0.05%,0.05%
cbBTC/USDC (86.0%),ethereum,$157.95,$146.33,$304.20,$24.01,$11.61,92.65%,0.08%,0.08%
ETH+/WETH (94.5%),ethereum,$97.66,$81.92,$102.25,$15.73,$15.73,83.89%,0.05%,0.04%
wstETH/WETH (96.5%),ethereum,$80.23,$67.15,$72.95,$16.83,$13.07,83.70%,0.02%,0.02%
WBTC/USDC (86.0%),ethereum,$80.03,$74.12,$168.25,$30.76,$5.91,92.61%,0.08%,0.08%
syrupUSDC/USDC (91.5%),ethereum,$70.19,$64.32,$81.79,$21.01,$5.88,91.63%,0.10%,0.09%
srUSD/rUSD (98.0%),ethereum,$65.89,$59.18,$61.81,$33.11,$6.71,89.82%,0.07%,0.07%
PT-cUSDO-19JUN2025/USDC (91.5%),ethereum,$60.40,$55.51,$75.55,$4.88,$4.88,91.91%,0.10%,0.09%
srUSD/USDC (91.5%),ethereum,$59.56,$54.29,$60.57,$10.60,$5.26,91.17%,0.09%,0.09%
PT-syrupUSDC-28AUG2025/USDC (86.0%),ethereum,$55.92,$51.07,$74.70,$11.83,$4.84,91.34%,0.10%,0.09%


In [20]:
def strip_utlization(market:str) -> str:
    match = re.search(r'.*\s', market)
    if match:
        return match.group()
    return ''

## Plot Morpho Markets

In [21]:
def plot_morpho_markets(df:pd.DataFrame) -> None:
    # Remove the utlization component from the market label
    stripped_markets = [strip_utlization(x) for x in df['market']]
    # Combine striped markets array with chains to add a chain to the market
    markets_chains = [x + y for x, y in zip(stripped_markets, df['chain'])]    

    # Plot bar chart with Suuply and Collateral assets
    # Numbers are rounded to 2 decimal points
    fig = go.Figure(data=[
        go.Bar(name='Supply Assets', x=markets_chains,
               y=[round(x,2) for x in df['supplyAssetsUsd'].values.tolist()]),
        go.Bar(name='Collateral Assets', x=markets_chains,
               y=[round(x,2) for x in df['collateralAssetsUsd'].values.tolist()])
    ])
    # Format the timestamp from the first item from the timestamp
    ts = time.strftime('%Y-%m-%d %H:%M:%S %Z', time.gmtime(df['timestamp'][0].item()))
    
    # Change the bar mode
    fig.update_layout(barmode='stack', title=dict(text=f'<b>Morpho Blues State as at {ts}</b>',
                      font=dict(size=20, color='cornflowerblue'), x=0.5, y=0.9, xanchor='center'),
                      xaxis=dict(title=dict(text='Markets (chain)', font_size=16), showticklabels=False),
                      yaxis=dict(title=dict(text='Value (mm) USD', font_size=16)), template='plotly_dark')
    fig.show()

In [22]:
# plot_morpho_markets(df=df)